In [ ]:
from qdrant_client import QdrantClient
from llama_index.core import VectorStoreIndex, PromptTemplate
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import LLMRerank

# 1. Bağlantılar
client = QdrantClient(url="http://localhost:6333")
#embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base")
llm = Ollama(
    model="llama3", 
    request_timeout=120.0,
    additional_kwargs={
        "keep_alive": 0
    }
)

# 2. Mevcut İndeksi Yükleme
vector_store = QdrantVectorStore(
    collection_name="bank_compliance_v1",
    client=client,
    enable_hybrid=True,
    batch_size=64,
    dense_vector_name="dense",
    sparse_vector_name="sparse"
)

index = VectorStoreIndex.from_vector_store(vector_store=vector_store, embed_model=embed_model)

# 3. Prompt Template
qa_prompt_tmpl = PromptTemplate(
    "SADECE SANA VERİLEN BAĞLAMDAKİ BİLGİLERİ KULLANARAK CEVAP VER.\n"
    "KESİNLİKLE TÜRKÇE DIŞINDA BİR DİL KULLANMA.\n"
    "Sen kıdemli bir Bankacılık Uyum Analistisin. Cevabını verirken mutlaka hangi maddeye dayandığını belirt.\n"
    "BAĞLAM:\n{context_str}\n"
    "SORU: {query_str}\n"
    "YANIT (SADECE TÜRKÇE): "
)

base_retriever = index.as_retriever(similarity_top_k=5)
#reranker = LLMRerank(choice_batch_size=20, top_n=5, llm=llm)

query_engine = RetrieverQueryEngine.from_args(
    base_retriever, 
    llm=llm,
    #node_postprocessors=[reranker]
)
query_engine.update_prompts({"response_synthesizer:text_qa_template": qa_prompt_tmpl})

/home/berk/finreg-navigator/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 199/199 [00:00<00:00, 1270.33it/s, Materializing param=pooler.dense.weight]                               
XLMRobertaModel LOAD REPORT from: intfloat/multilingual-e5-base
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Fetching 5 files: 100%|██████████| 5/5 [00:22<00:00,  4.40s/it]


In [4]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.run_config import RunConfig
from datasets import Dataset
from langchain_ollama import ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings

# Ragas için LangChain nesnelerini Ragas sarmalayıcılarıyla sar
evaluator_llm = LangchainLLMWrapper(ChatOllama(model="llama3", keep_alive=0))
evaluator_embeddings = LangchainEmbeddingsWrapper(HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base"))

questions = [
    "Açık bankacılık servisleri nedir?",
    "Bu Yönetmeliğin amacı nedir?",
    "Hassas veri tanımı neleri kapsar?",
    "Banka yönetim kurulunun temel sorumluluğu nedir?",
    "BS Strateji Komitesi yılda en az kaç kez toplanır ve kime rapor sunar?",
    "Bilgi sistemleri (BS) nedir?",
    "BS Yönlendirme Komitesinin sorumlulukları nelerdir?",
    "BS politika ve prosedürleri ne sıklıkla gözden geçirilmelidir?",
    "BS politikaları kim tarafından onaylanır?",
    "Elektronik bankacılık hizmetleri neleri kapsar?"
]

ground_truths = [
    "Müşterilerin ya da müşteriler adına hareket eden tarafların API, web servis, dosya transfer protokolü gibi yöntemlerle bankanın sunduğu finansal servislere uzaktan erişerek bankacılık işlemlerini gerçekleştirebildikleri veya gerçekleştirilmesi için bankaya talimat verebildikleri elektronik dağıtım kanalını ifade eder.",
    "Bankaların faaliyetlerinin ifasında kullandıkları bilgi sistemlerinin yönetimi ile elektronik bankacılık hizmetlerinin sunulmasında ve bunlara ilişkin risklerin yönetiminde esas alınacak asgari usul ve esaslar ile tesis edilmesi gereken bilgi sistemleri kontrollerini düzenlemektir.",
    "Kimlik doğrulamada kullanılan veriler başta olmak üzere; müşteriye ait olan, çeşitli sebeplerle bankaca muhafaza edilen ve üçüncü kişilerce ele geçirilmesi halinde, bu kişilerin müşteri olan kişilerle ayırt edilebilme mekanizmalarının zarar göreceği ve dolandırıcılık ya da müşteriler adına sahte işlem yapılmasına imkân verebilecek nitelikteki verileri ifade eder.",
    "Bilgi sistemlerinin yönetimini kurumsal yönetim uygulamalarının bir parçası olarak ele almakla, bilgi sistemlerinin doğru yönetimi için gerekli finansman ve insan kaynağını tahsis etmekle, bilgi varlıklarının gizliliği, bütünlüğü ve erişilebilirliğini sağlamak amacıyla bilgi sistemleri üzerinde etkin kontrollerin tesis edilmesini sağlamakla ve gelişen yeni teknolojileri de göz önünde bulundurarak bilgi sistemlerinin kullanımından kaynaklanan risklerin yönetilmesi için etkin bir gözetim yürütmekle sorumludur.",
    "BS strateji planının düzgün bir şekilde uygulanıp uygulanmadığını gözden geçirmek ve önemli BS yatırım kararlarını değerlendirmek üzere yılda en az iki defa bir araya gelir ve yılda en az bir defa yönetim kuruluna rapor sunar.",
    "Bilginin toplanması, işlenmesi, saklanması, dağıtımı ve kullanımına yönelik insan kaynağı, operasyonel faaliyetler ve süreçler ile bunlarla etkileşim içinde bulunan bilgi teknolojilerini ifade eder.",
    "BS yatırımlarının ve projelerinin öncelik sırasını belirlemek, devam eden BS projelerinin durumunu takip etmek, projeler arasındaki kaynak çatışmalarını çözüme kavuşturmak, BS mimarisi ve BS projelerinin mevzuata uyumluluğunu sağlamak üzere gerekli yönlendirmeleri yapmak ve BS servislerine ilişkin hizmet seviyelerini izlemekten sorumludur.",
    "BS politika, prosedür ve süreç dokümanları yılda en az bir defa gözden geçirilir ve gerekli güncellemeler yapılır.",
    "BS politikaları yönetim kurulu tarafından onaylanır.",
    "İnternet bankacılığı, mobil bankacılık, telefon bankacılığı, açık bankacılık servisleri ile ATM ve kiosk cihazları gibi müşterilerin, uzaktan bankacılık işlemlerini gerçekleştirebildikleri veya gerçekleştirilmesi için bankaya talimat verebildikleri her türlü elektronik dağıtım kanalını ifade eder."
]

results_data = {"question": [], "answer": [], "contexts": [], "ground_truth": []}

for i, q in enumerate(questions):
    response = query_engine.query(q)
    results_data["question"].append(q)
    results_data["answer"].append(response.response)
    results_data["contexts"].append([n.get_content() for n in response.source_nodes])
    results_data["ground_truth"].append(ground_truths[i])

dataset = Dataset.from_dict(results_data)

eval_result = evaluate(
    dataset,
    metrics=[faithfulness, answer_relevancy], #context_precision
    llm=evaluator_llm,
    embeddings=evaluator_embeddings,
    run_config=RunConfig(timeout=240, max_workers=1)
)

/tmp/ipykernel_23980/4185188469.py:2: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import faithfulness, answer_relevancy, context_precision
/tmp/ipykernel_23980/4185188469.py:2: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import faithfulness, answer_relevancy, context_precision
/tmp/ipykernel_23980/4185188469.py:2: DeprecationWarning: Importing context_precision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_precision
  from ragas.metrics import faithfulness, answer_relevancy, context

In [5]:
scores_df = eval_result.to_pandas()
metrics_dict = scores_df.select_dtypes(include="number").mean().to_dict()
print("Metrikler:", metrics_dict)

Metrikler: {'faithfulness': 0.6388888888888888, 'answer_relevancy': 0.8570712012356315}


In [ ]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Banking_RAG_Ingestion")

# EvaluationResult → MLflow'un beklediği düz dict'e çevir
scores_df = eval_result.to_pandas()
metrics_dict = scores_df.select_dtypes(include="number").mean().to_dict()

with mlflow.start_run(run_name="Llama3_v1"):
    # Parser ismini SentenceSplitter olarak güncelledik
    mlflow.log_params({"top_k": 5, "model": "llama3", "Parser": "SentenceSplitter", "Embedding": "multilingual-e5-base"})
    mlflow.log_metrics(metrics_dict)
    print("Değerlendirme tamamlandı. Sonuçlar MLflow'da!")
    print("Metrikler:", metrics_dict)

2026/02/26 21:43:55 INFO mlflow.tracking.fluent: Experiment with name 'Banking_RAG_Ingestion' does not exist. Creating a new experiment.


Değerlendirme tamamlandı. Sonuçlar MLflow'da!
Metrikler: {'faithfulness': 0.6388888888888888, 'answer_relevancy': 0.8570712012356315}
🏃 View run Llama3_Rerank_v1 at: http://localhost:5000/#/experiments/1/runs/cb851b6f56e24428900318306cd1d473
🧪 View experiment at: http://localhost:5000/#/experiments/1


: 